<!-- Basic steps for FEM:
1. Find the physical problem
2. Describe the mathematical model
    1. Governing equations (PDEs, boundary conditions, initial conditions)
    2. Constant parameters , source functions
    3. Define the unknown quantities to solve for
3. Create FE space to find the unknown quantities in.
    1. Create mesh
        1. Mesh density & mesh profile
        2. Type of FE spaces
        3. Conformity of the FE functions 
4. Writing the weak form of the problem.
5. Solving the problem.
6. Iterating for different mesh sizes until solution converges. -->

# ***Steady-State Convection-Diffusion Problem***

## Problem Description
We aim to solve the steady-state convection-diffusion equation over a square domain $\Omega = [0, 1] \times [0, 1]$, with specified boundary conditions. This equation models the transport of a scalar field $u(x, y)$ under the influence of advection, diffusion, and a source term.

---

## Governing Equation
$$
-\nabla \cdot (D \nabla u) + \mathbf{v} \cdot \nabla u + \sigma u = f \quad \text{in } \Omega,
$$
where:
- $u(x, y)$: scalar field (e.g., temperature, concentration, etc.).
- $D > 0$: diffusion coefficient (assume constant for simplicity).
- $\mathbf{v} = (v_x, v_y)$: convection velocity field (assume $\mathbf{v} = (1, 0)$, representing flow in the $x$-direction).
- $\sigma \geq 0$: reaction coefficient (assume $\sigma = 1$).
- $f(x, y)$: source term (e.g., $f(x, y) = \sin(\pi x)\sin(\pi y)$).

---

## Boundary Conditions
1. **Dirichlet boundary condition**:
   $$
   u = 0 \quad \text{on the inflow boundary } \Gamma_{\text{in}} = \{x = 0\}.
   $$
2. **Neumann boundary condition**:
   $$
   -D \frac{\partial u}{\partial n} = 0 \quad \text{on the outflow boundary } \Gamma_{\text{out}} = \{x = 1\}.
   $$
3. **Mixed boundary conditions**:
   - **No-flux condition**: 
     $$
     -D \frac{\partial u}{\partial n} = 0 \quad \text{on the top boundary } \Gamma_{\text{top}} = \{y = 1\}.
     $$
   - **No-flux condition**: 
     $$
     -D \frac{\partial u}{\partial n} = 0 \quad \text{on the bottom boundary } \Gamma_{\text{bottom}} = \{y = 0\}.
     $$

---

## Domain
$$
\Omega = [0, 1] \times [0, 1].
$$

---

## Parameters
- $D = 0.01$.
- $\mathbf{v} = (1, 0)$.
- $\sigma = 1$.
- $f(x, y) = \sin(\pi x) \sin(\pi y)$.

---


In [111]:
using Gridap
using Gridap.Geometry

# Parameters and functions which we can directly define
D = 0.01
v = VectorValue(1.0,0.0)
σ = 1
f(x) = sin(π*x[1])*sin(π*x[2])

# Define the mesh
L = 1.0
n = 10
domain = (0.0, L, 0.0, L)
partition = (n, n)
model = CartesianDiscreteModel(domain, partition)

# Get the labels
labels = get_face_labeling(model)

# Label the boundaries
add_tag_from_tags!(labels, "Dirichlet", [5]) # the left entity (number 5) is Dirichlet
add_tag_from_tags!(labels, "Neumann", [6,7,8]) # the lower entity (number 6), right entity (number 7) and the upper entity (number 8) are Neumann

Ω = Triangulation(model)
Γ_D = Boundary(Ω,tags="Dirichlet")
Γ_N = Boundary(Ω,tags="Neumann")

# Define the finite element space
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)
V0 = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags="Dirichlet")
U0 = TrialFESpace(V0, 0.0)

# Define the integration measure
degree = 2*order
dΩ = Measure(Ω,degree)
dΓ_D  = Measure(Γ_D,degree)
dΓ_N = Measure(Γ_N,degree)
n_ΓN = get_normal_vector(Γ_N)
n_ΓD = get_normal_vector(Γ_D)

# Define the bilinear form
a(u,w) = ∫( D*(∇(u)⋅∇(w)) + (v⋅∇(u))*w + w*σ*u)dΩ
b(w) = ∫(f*w)dΩ 

b (generic function with 1 method)

In [112]:
op = AffineFEOperator(a,b,U0,V0)
ls = LUSolver()
uh = solve(solver,op)

SingleFieldFEFunction():
 num_cells: 100
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 3353973341585016191

In [113]:
writevtk(Ω,"results",cellfields=["uh"=>uh])

(["results.vtu"],)